# 1. Imports and Loading the Network

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
G = nx.read_edgelist("data/sbb.edgelist", delimiter=";", create_using=nx.Graph)

connected_comp = nx.connected_components(G)
max_connected_comp = max(connected_comp)
G = G.subgraph(max_connected_comp)

# 2. Benchmark: Iteration Time, Silhouette Index and Modularity

In [3]:
from sklearn.metrics import silhouette_score
import time
import os

# create directory to save benchmarks results
if not os.path.exists("data/benchmarks"):
    os.makedirs("data/benchmarks")


def multi_iter_benchmark(G, community_detec_func, iters=10, time_threshold=12, leiden=False):
    times = []
    scores = []
    modularities = []
    
    for _ in range(iters):
        partitions = community_detec_func(G)
        t, s, m = single_iter_benchmark(G, partitions, time_threshold, leiden)
        times.append(t)
        scores.append(s)
        modularities.append(m)

    max_len_iteration = max([len(t) for t in times])
    
    for (t, s, m) in zip(times, scores, modularities):
        while len(t) < max_len_iteration:
            t.append(np.nan)
            s.append(np.nan)
            m.append(np.nan)

    mean_times = np.nanmean(times, axis=0)
    mean_scores = np.nanmean(scores, axis=0)
    mean_modularities = np.nanmean(modularities, axis=0)

    return mean_times, mean_scores, mean_modularities

# iteration benchmarking function
# you must give it a partition scheme before entering the function
# time_threshold is the maximum number of hours the benchmark should run for. Useful for Girvan-Newman
def single_iter_benchmark(G, partitions, time_threshold=12, leiden=False):
    timeout = time.time() + 60 * 60 * time_threshold

    i_time = []
    i_silhouette = []
    i_modularity = []
    
    while True:
        # Timeout necessary for Girvan Newman
        if time.time() > timeout:
            break
            
        start = time.time()
        communities = next(partitions, 'finished')
        stop = time.time()
        
        if communities == 'finished':
            break
        
        time_diff = stop - start
        i_time.append(time_diff)

        # Necessary for leiden: convert node ids to node names
        if leiden == True:
            communities = convert_leiden(G, communities)
        
        i_silhouette.append(silhouette_fn(G, communities))
        i_modularity.append(nx.community.modularity(G, communities))
        
        final_communities = communities
        
    return i_time, i_silhouette, i_modularity


# silhouette score function
def silhouette_fn(G, found_communities):
    # make a dictionary with node as key and final community as value
    node_to_community = {node: comm_idx for comm_idx, community in enumerate(found_communities) for node in community}
    # list all final communities by indexing with keys
    comm_labels = [node_to_community[node] for node in G.nodes()]
    
    # calc silhouette score by inputting graph as an array and the communities as labels
    sil_score = silhouette_score(nx.to_numpy_array(G), comm_labels, metric='euclidean', random_state=42)
    return sil_score

## 2.1 Louvain

In [4]:
t, s, m = multi_iter_benchmark(G, nx.community.louvain_partitions)

louvain_bench = pd.DataFrame(zip(t, s, m), columns=["time", "silhouette", "modularity"])
#louvain_bench.to_csv("./data/benchmarks/louvain.csv", index=False)
louvain_bench

,time,silhouette,modularity
0,0.184504,-0.063911,0.560196
1,0.049644,-0.021649,0.818520
2,0.017919,-0.058597,0.896203
3,0.005718,-0.068911,0.906182
4,0.002111,-0.068969,0.906309


## 2.2 Louvain Implementation

In [ ]:
from louvain import louvain_generator

G = nx.read_edgelist("data/sbb.edgelist", delimiter=";", create_using=nx.Graph)
connected_comp = nx.connected_components(G)
max_connected_comp = max(connected_comp)
sub_G = G.subgraph(max_connected_comp)
G = nx.Graph(sub_G)

louvain_imp_partitions = louvain_generator(G)
t, s, m = iter_benchmark(G, louvain_imp_partitions)

louvain_imp_bench = pd.DataFrame(zip(t, s, m), columns=["time", "silhouette", "modularity"])
louvain_imp_bench.to_csv("./data/benchmarks/louvain_imp.csv", index=False)

## 2.3 Girvan-Newman

In [ ]:
gn_partitions = nx.community.girvan_newman(G)
t, s, m = iter_benchmark(G, gn_partitions, time_threshold=8)

gn_bench = pd.DataFrame(zip(t, s, m), columns=["time", "silhouette", "modularity"])
gn_bench.to_csv("./data/benchmarks/girvan-newman.csv", index=False)

## 2.4 Leiden

In [ ]:
import leidenalg as la
import igraph as ig


def convert_leiden(G, partition):
    idx2name = {idx: name for idx, name in enumerate(list(G.nodes))}
    
    communities = []
    for comm in partition:
        communities.append({idx2name[n] for n in comm})
        
    return communities


def leiden_gen(G):
    graph = ig.Graph.from_networkx(G, vertex_attr_hashable='name')

    # create the partition with every node as its community
    partition = la.ModularityVertexPartition(graph)
    optimiser = la.Optimiser()
        
    diff = 1
    while diff > 0:
        diff = optimiser.optimise_partition(partition)
        yield partition

In [ ]:
t, s, m = multi_iter_benchmark(G,leiden_gen, leiden=True)

leiden_bench = pd.DataFrame(zip(t, s, m), columns=["time", "silhouette", "modularity"])

In [ ]:
leiden_bench

In [ ]:
leiden_bench = pd.DataFrame(zip(t, s, m), columns=["time", "silhouette", "modularity"])
leiden_bench.to_csv("./data/benchmarks/leiden.csv", index=False)

In [ ]:
plot_bench_results([louvain_bench, leiden_bench], ["Louvain", "Leiden"])

# 3. Evaluation of the Community Detection Algorithms

In [ ]:
def plot_bench_results(df_list, labels):
    fig, axs = plt.subplots(nrows=1, ncols=len(df_list[0].columns), figsize=(18, 6), layout="tight")
    # iterate over the columns
    for i, col in enumerate(df_list[0].columns):
        # iterate over the dataframe: all in the same plot
        for df, label in zip(df_list, labels):
            axs[i].plot(df[col], label=label)
        
        axs[i].set_xlabel('Iteration')
        axs[i].set_ylabel(col)
        axs[i].set_title(f'{col} v. Iteration')
            # force int on x-axis
        axs[i].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        axs[i].legend()
    
    plt.savefig("community-detection-benchmarks.png")
    plt.show()

In [ ]:
louvain_bench = pd.read_csv("data/benchmarks/louvain.csv")
louvain_imp_bench = pd.read_csv("data/benchmarks/louvain_imp.csv")
gn_bench = pd.read_csv("data/benchmarks/girvan-newman.csv")
leiden_bench = pd.read_csv("data/benchmarks/leiden.csv")

algorithm_data = [louvain_bench, louvain_imp_bench, leiden_bench, gn_bench[:9]]
algorithm_name = ["Louvain", "Louvain Implementation", "Leiden", "Girvan-Newman"]

plot_bench_results(algorithm_data, algorithm_name)

In [ ]:
# we should be able to safely remove this
# kept just in case
def eval_plots(*datasets, tick_int):
    for i, (data, label, ylabel) in enumerate(datasets):
        for d, l in zip(data, label): 
            ax[i].plot(d, label=l)
        ax[i].set_xlabel('Iterations')
        ax[i].set_ylabel(ylabel)
        ax[i].set_title(f'{ylabel} vs Iterations')
        ax[i].xaxis.set_major_locator(ticker.MultipleLocator(tick_int))
        ax[i].legend()
        
    plt.tight_layout()
    plt.show()




#fig, ax = plt.subplots(1, 3, figsize=(12, 6))

# Plotting with variable names as legend labels
#eval_plots(((nx_lou_m_times, nx_gn_m_times[:nx_lou_times.shape[1]]), ('NetworkX Louvain', 'NetworkX Girvan Newman'), 'Time (s)'),
            #((nx_lou_m_sil, nx_gn_m_sil[:nx_lou_times.shape[1]]), ('NetworkX Louvain', 'NetworkX Girvan Newman'), 'Silhouette Score'),
            #((nx_lou_m_mod, nx_gn_m_mod[:nx_lou_times.shape[1]]), ('NetworkX Louvain', 'NetworkX Girvan Newman'), 'Modularity Score'), 
           #tick_int=1)


# Just Girvan Newman plots
# read nx gn csvs
nx_gn_times = pd.read_csv("data/communities/nx_gn_iteration_times.csv")
nx_gn_m_times = nx_gn_times.mean(axis='rows')

nx_gn_sil = pd.read_csv("data/communities/nx_gn_silhouette_scores_per_iteration.csv")
nx_gn_m_sil = nx_gn_sil.mean(axis='rows')

nx_gn_mod = pd.read_csv("data/communities/nx_gn_modularity_scores_per_iteration.csv")
nx_gn_m_mod = nx_gn_mod.mean(axis='rows')

fig, ax = plt.subplots(3, 1, figsize=(12, 12))

eval_plots(((nx_gn_m_times,), ('NetworkX Girvan Newman',), 'Time (s)'),
            ((nx_gn_m_sil,), ('NetworkX Girvan Newman',), 'Silhouette Score'),
            ((nx_gn_m_mod,), ('NetworkX Girvan Newman',), 'Modularity Score'),
           tick_int=200)
